In [1]:
import openpyxl
import selenium
import pandas as pd
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException  

In [2]:

# Load the input file
wb = openpyxl.load_workbook(r'C:\Users\USER\Downloads\Input.xlsx')
ws = wb.active

# Iterate through the rows and extract the URLs
urls = []
for row in ws.iter_rows(values_only=True):
    for cell in row:
        if isinstance(cell, str) and cell.startswith('http'):
            urls.append(cell)


In [3]:
urls

['https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/',
 'https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/',
 'https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/',
 'https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/',
 'https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/',
 'https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/',
 'https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/',
 'https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/',
 'https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/',
 'https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/',
 'https://insights.blackcoffer.com/how-to-protect-fu

In [27]:
driver = webdriver.Chrome()

In [98]:
title = []
text = []
# extract the data through urls
for i in urls:
    driver.get(i)
    time.sleep(3)
    
    try:
        job = driver.find_element_by_xpath("//h1[@class='entry-title']")
        title.append(job.text)
        
    except NoSuchElementException:
        title.append('Nan')
        
    try:
        loc = driver.find_element_by_xpath("//*[starts-with(@id, 'post-')]/div[3]")
        loc.find_elements_by_xpath(".//p[position() >= 2 and position() <= 32]")
        text.append(loc.text)
    except NoSuchElementException:
        text.append('Nan')

In [101]:
len(title)

114

In [102]:
len(text)

114

In [103]:
df = pd.DataFrame({})
df["Title"] = title
df["Text"] = text

In [104]:
df.to_csv("Blackoffer.csv")

Load dataset

In [4]:
df = pd.read_csv("Blackoffer.csv",)
df

Unnamed: 0                                              Title  \
0             0       AI in healthcare to Improve Patient Outcomes   
1             1   What if the Creation is Taking Over the Creator?   
2             2  What Jobs Will Robots Take From Humans in The ...   
3             3  Will Machine Replace The Human in the Future o...   
4             4                Will AI Replace Us or Work With Us?   
..          ...                                                ...   
109         109                            Blockchain for Payments   
110         110                            The future of Investing   
111         111                   Big Data Analytics in Healthcare   
112         112      Business Analytics In The Healthcare Industry   
113         113  Challenges and Opportunities of Big Data in He...   

                                                  Text  
0    Introduction\n“If anything kills over 10 milli...  
1    Human minds, a fascination in itself carrying ...  
2    Introduction\nAI is rapidly evolving in the em...  
3    “Anything that could give rise to smarter-than...  
4    “Machine intelligence is the last invention th...  
..                                                 ...  
109  Reconciling with the financial realities of an...  
110  What Is an Investment?\nAn investment is a res...  
111  Quality and affordable healthcare is a vision ...  
112  Analytics is a statistical scientific process ...  
113  Big Data\nTo begin with I shall first like to ...  

[114 rows x 3 columns]

preprocessing

In [5]:
# convert all message in to lower cases
df["Text"] = df["Text"].str.lower()

In [6]:
# remove punctuation
df["Text"]= df["Text"].str.replace(r'[^\w\d\s]',' ')

#replace whitespace between terms with a single space
df["Text"] = df["Text"].str.replace(r'\s+',' ')

#remove leading and trailing whitespace
df["Text"] = df["Text"].str.replace(r'^\s+|\s+?$', ' ')

Import Libraries

In [7]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')
from textblob import TextBlob
#pip install textstat
import textstat
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize

Positive score

In [8]:
# initialize SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [9]:
# create new column for positive scores
df['POSITIVE SCORE'] = df['Text'].apply(lambda x: sia.polarity_scores(x)['pos'])

Negative score

In [10]:
# Define a function to compute the sentiment polarity score of each text column
def get_negative_score(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity if blob.sentiment.polarity < 0 else 0

In [11]:
# Apply the function to your text column to get the negative score column
df['NEGATIVE SCORE'] = df['Text'].apply(get_negative_score)

Polarity score

In [12]:
# Create an empty list to store the polarity scores
polarity_scores = []

# Define a function to get polarity score for each text in the dataset
def get_polarity_score(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

# Iterate through the text column in your dataset and get the polarity score for each text
for text in df['Text']:
    score = get_polarity_score(text)
    polarity_scores.append(score)

# Add the polarity score column to your dataset
df['POLARITY SCORE'] = polarity_scores

subjective score

In [13]:
# Create a function to calculate the subjectivity score
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Apply the function to your text column
df['SUBJECTIVITY SCORE'] = df['Text'].apply(get_subjectivity)

Functions

In [14]:
# Define a function to calculate the percentage of complex words
def percentage_complex_words(text):
    # Get all the words
    words = word_tokenize(text)
    # Remove the stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word.lower() in stop_words]
    # Get the Porter stem of each word
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    # Count the number of complex words
    complex_words = [word for word in words if textstat.syllable_count(word) > 2]
    complex_word_count = len(complex_words)
    # Return the percentage of complex words
    return complex_word_count / len(words) * 100

# Define a function to calculate the FOG index
def fog_index(text):
    # Get the average number of words per sentence
    sentences = sent_tokenize(text)
    words_per_sentence = sum(len(word_tokenize(sentence)) for sentence in sentences) / len(sentences)
    # Get the percentage of complex words
    percentage_complex = percentage_complex_words(text)
    # Calculate the FOG index
    fog_index = 0.4 * (words_per_sentence + percentage_complex)
    return fog_index

# Define a function to count the number of personal pronouns
def personal_pronouns(text):
    pronouns = ['i', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves']
    words = word_tokenize(text.lower())
    return len([word for word in words if word in pronouns])

# Define a function to calculate the average word length
def avg_word_length(text):
    words = word_tokenize(text.lower())
    return sum(len(word) for word in words) / len(words)

In [15]:
# Add the columns
df['AVG SENTENCE LENGTH'] = df['Text'].apply(lambda x: len(sent_tokenize(x)))
df['PERCENTAGE OF COMPLEX WORDS'] = df['Text'].apply(percentage_complex_words)
df['FOG INDEX'] = df['Text'].apply(fog_index)
df['AVG NUMBER OF WORDS PER SENTENCE'] = df['Text'].apply(lambda x: len(word_tokenize(x)) / len(sent_tokenize(x)))
df['COMPLEX WORD COUNT'] = df['Text'].apply(lambda x: len([word for word in word_tokenize(x) if textstat.syllable_count(word) > 2]))
df['WORD COUNT'] = df['Text'].apply(lambda x: len(word_tokenize(x)))
df['SYLLABLE PER WORD'] = df['Text'].apply(lambda x: textstat.syllable_count(x) / len(word_tokenize(x)))
df['PERSONAL PRONOUNS'] = df['Text'].apply(personal_pronouns)
df['AVG WORD LENGTH'] = df['Text'].apply(avg_word_length)

In [16]:
df.head()

Unnamed: 0                                              Title  \
0           0       AI in healthcare to Improve Patient Outcomes   
1           1   What if the Creation is Taking Over the Creator?   
2           2  What Jobs Will Robots Take From Humans in The ...   
3           3  Will Machine Replace The Human in the Future o...   
4           4                Will AI Replace Us or Work With Us?   

                                                Text  POSITIVE SCORE  \
0  introduction if anything kills over 10 million...           0.144   
1  human minds a fascination in itself carrying t...           0.150   
2  introduction ai is rapidly evolving in the emp...           0.075   
3   anything that could give rise to smarter than...           0.147   
4   machine intelligence is the last invention th...           0.153   

   NEGATIVE SCORE  POLARITY SCORE  SUBJECTIVITY SCORE  AVG SENTENCE LENGTH  \
0             0.0        0.136121            0.460677                    1   
1             0.0        0.073755            0.428819                    1   
2             0.0        0.111470            0.483201                    1   
3             0.0        0.137689            0.481786                    1   
4             0.0        0.054452            0.504035                    1   

   PERCENTAGE OF COMPLEX WORDS   FOG INDEX  AVG NUMBER OF WORDS PER SENTENCE  \
0                    11.196581  734.878632                            1826.0   
1                     8.912189  581.564875                            1445.0   
2                    13.844622  691.937849                            1716.0   
3                    10.537870  667.815148                            1659.0   
4                    11.881188  720.752475                            1790.0   

   COMPLEX WORD COUNT  WORD COUNT  SYLLABLE PER WORD  PERSONAL PRONOUNS  \
0                 364        1826           1.748083                  4   
1                 183        1445           1.499654                 11   
2                 340        1716           1.732517                  3   
3                 231        1659           1.529234                 24   
4                 274        1790           1.601117                 19   

   AVG WORD LENGTH  
0         5.553122  
1         4.728720  
2         5.362471  
3         4.774563  
4         4.958659

In [17]:
df_excel= pd.read_excel(r"C:\Users\USER\Downloads\Input.xlsx")
df_excel

URL_ID                                                URL
0        37  https://insights.blackcoffer.com/ai-in-healthc...
1        38  https://insights.blackcoffer.com/what-if-the-c...
2        39  https://insights.blackcoffer.com/what-jobs-wil...
3        40  https://insights.blackcoffer.com/will-machine-...
4        41  https://insights.blackcoffer.com/will-ai-repla...
..      ...                                                ...
109     146  https://insights.blackcoffer.com/blockchain-fo...
110     147  https://insights.blackcoffer.com/the-future-of...
111     148  https://insights.blackcoffer.com/big-data-anal...
112     149  https://insights.blackcoffer.com/business-anal...
113     150  https://insights.blackcoffer.com/challenges-an...

[114 rows x 2 columns]

In [22]:
df.columns

Index(['Unnamed: 0', 'Title', 'Text', 'POSITIVE SCORE', 'NEGATIVE SCORE',
       'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],
      dtype='object')

In [41]:
columns =['POSITIVE SCORE', 'NEGATIVE SCORE',
       'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']
df_new = pd.concat([df_excel,df[columns]],axis=1)

In [42]:
df_new

URL_ID                                                URL  \
0        37  https://insights.blackcoffer.com/ai-in-healthc...   
1        38  https://insights.blackcoffer.com/what-if-the-c...   
2        39  https://insights.blackcoffer.com/what-jobs-wil...   
3        40  https://insights.blackcoffer.com/will-machine-...   
4        41  https://insights.blackcoffer.com/will-ai-repla...   
..      ...                                                ...   
109     146  https://insights.blackcoffer.com/blockchain-fo...   
110     147  https://insights.blackcoffer.com/the-future-of...   
111     148  https://insights.blackcoffer.com/big-data-anal...   
112     149  https://insights.blackcoffer.com/business-anal...   
113     150  https://insights.blackcoffer.com/challenges-an...   

     POSITIVE SCORE  NEGATIVE SCORE  POLARITY SCORE  SUBJECTIVITY SCORE  \
0             0.144             0.0        0.136121            0.460677   
1             0.150             0.0        0.073755            0.428819   
2             0.075             0.0        0.111470            0.483201   
3             0.147             0.0        0.137689            0.481786   
4             0.153             0.0        0.054452            0.504035   
..              ...             ...             ...                 ...   
109           0.128             0.0        0.028516            0.370661   
110           0.117             0.0        0.070117            0.387202   
111           0.090             0.0        0.039977            0.414350   
112           0.174             0.0        0.197317            0.540860   
113           0.187             0.0        0.122248            0.467322   

     AVG SENTENCE LENGTH  PERCENTAGE OF COMPLEX WORDS   FOG INDEX  \
0                      1                    11.196581  734.878632   
1                      1                     8.912189  581.564875   
2                      1                    13.844622  691.937849   
3                      1                    10.537870  667.815148   
4                      1                    11.881188  720.752475   
..                   ...                          ...         ...   
109                    1                     7.936508  382.774603   
110                    1                     8.242304  647.296922   
111                    1                    12.463768  476.185507   
112                    1                    14.814815  295.525926   
113                    1                     9.317804  424.127121   

     AVG NUMBER OF WORDS PER SENTENCE  COMPLEX WORD COUNT  WORD COUNT  \
0                              1826.0                 364        1826   
1                              1445.0                 183        1445   
2                              1716.0                 340        1716   
3                              1659.0                 231        1659   
4                              1790.0                 274        1790   
..                                ...                 ...         ...   
109                             949.0                 145         949   
110                            1610.0                 228        1610   
111                            1178.0                 216        1178   
112                             724.0                 168         724   
113                            1051.0                 165        1051   

     SYLLABLE PER WORD  PERSONAL PRONOUNS  AVG WORD LENGTH  
0             1.748083                  4         5.553122  
1             1.499654                 11         4.728720  
2             1.732517                  3         5.362471  
3             1.529234                 24         4.774563  
4             1.601117                 19         4.958659  
..                 ...                ...              ...  
109           1.591149                 13         5.144362  
110           1.570186                 18         4.944099  
111           1.649406                  2         4.9949

In [43]:
df_new.to_excel("Output Data Structure.xlsx", index=False)